# 8.1 维度的诅咒

* 高维数据集很大可能是非常稀疏的：大多数训练实例可能彼此之间相距很远
* 训练集的维度越高，过拟合的风险就越大

# 8.2 降维的主要方法

## 8.2.1 投影

所有训练实例都位于(或接近于)高维空间的低维子空间内

投影并不只是降低尺寸的最佳方法。在许多情况下，子空间可能会发生扭曲和转动。

## 8.2.2 流形学习

通过对训练实例所在的流形进行建模  
依赖于流形假设：大多数现实世界的高维数据集都接近于低维流形  
在训练模型之前降低训练集的维度肯定可以加快训练速度，但这并不总是会导致更好或更简单的解决方案

# 8.3 PCA

最流行的降维算法  
首先识别最靠近数据的超平面，然后将数据投影到其上

## 8.3.1 保留差异性

将训练集投影到低维超平面之前需要选择正确的超平面  
选择保留最大差异性的轴看起来比较合理  
比较原始数据集与其轴上的投影之间的均方距离，使这个最小的轴是最合理的选择

## 8.3.2 主要成分

主成分分析可以在训练集中识别出哪条轴对差异性的贡献度最高   
第i个轴称为数据的第i个主要成分(PC)  
对于每个主要成分，PCA都找到一个指向PC方向的零中心单位向量

奇异值分解(SVD): 
将训练集矩形$X$分解为三个矩阵${U∑V}^T$的矩阵乘法，其中$V$包含定义所有主要成分的单位向量

In [3]:
# 使用NumPy的svd()函数来获取训练集的所有主要成分，然后提取定义前两个PC的两个单位向量

import numpy as np
from sklearn.datasets import make_swiss_roll

X, t = make_swiss_roll(n_samples=2000, noise=0.1)

X_centered = X - X.mean(axis=0)     # 居中数据
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]


## 8.3.3 向下投影到d维度

确定所有主要成分后，将数据集投影到前d个主要成分定义的超平面上，从而将数据集的维度降低到d维

将训练集投影到d维度
$$X{_{d-proj}}=XW{_d}$$  
矩阵$W_d$定义为包含$V$的前$d$列的矩阵

In [4]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

## 8.3.4 使用Scikit-Learn

In [6]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

将PCA转换器拟合到数据集后，其components_属性是$W_d$的转置  
定义第一个主成分的单位向量等于pca.components_.T[:, 0]

## 8.3.5 可解释方差比